In [1]:
%run 2.Navigation_FAISS_Index.ipynb

## Get Context from Retriever

In [14]:
from typing import List

def convert_to_context(docs: List[Document]):
    context = ""
    for doc in docs:
        content, id = doc.page_content, doc.metadata["id"]
        context += f"{id} - {content}\n###########\n"

    return context

In [15]:
results = retriever.invoke("Take me to login page")
results[0]

Document(id='cc794c58-08bc-4fe8-b138-c319c5302fa7', metadata={'id': 1}, page_content='Directs the user to the login page to authenticate and access the application.')

In [16]:
context = convert_to_context(docs=results)

In [17]:
print(context)

1 - Directs the user to the login page to authenticate and access the application.
###########
9 - Takes the user to the user list page to view and manage user profiles.
###########
17 - Navigates to the user preview page to review user details before saving.
###########
16 - Directs to the page for adding a new user to the system.
###########



## Design Prompt

In [30]:
RAG_PROMPT = """
You are an navigation assistant. Compare the user query with the context to identify the most appropriate navigation.
Each navigation option is separated by #####. Identify the most similar navigation.
Return only the ID for the most similar navigation and nothing else.
Query: {query}
Context: {context}
ID:
"""

In [31]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(RAG_PROMPT)

In [32]:
rag_chain = template | chat_model

In [37]:
QUERY = "show issue screen"

results = retriever.invoke(QUERY)
context = convert_to_context(docs=results)

print(results)
rag_chain.invoke({"query": QUERY, "context": context}).content

[Document(id='35da70b4-8a4d-477e-ada3-d28946d9b167', metadata={'id': 24}, page_content='Takes the user to the issue preview screen to view details of a specific task or issue.'), Document(id='7f1c8982-71ac-4057-9e1f-a468eb5983b7', metadata={'id': 23}, page_content='Navigates to the issue screen to view a list of reported tasks or issues.'), Document(id='65ecc77e-804c-435a-a21f-18a958f52b45', metadata={'id': 25}, page_content='Directs to the issue add or edit screen to create or modify an issue.'), Document(id='31360829-c408-4f6c-9114-f75c08ca6016', metadata={'id': 26}, page_content='Navigates to the issue filter view to refine the list of tasks or issues.')]


'The most similar navigation is 23.'

[Document(id='5735f190-8364-4bf8-b756-f8b9a96069c2', metadata={'id': 40}, page_content='Directs to the quick scan summary screen to view scan results on the mobile app.'),
 Document(id='35da70b4-8a4d-477e-ada3-d28946d9b167', metadata={'id': 24}, page_content='Takes the user to the issue preview screen to view details of a specific task or issue.'),
 Document(id='9aac8a84-1253-4a04-98ed-e20ac1d2d30f', metadata={'id': 17}, page_content='Navigates to the user preview page to review user details before saving.'),
 Document(id='7f1c8982-71ac-4057-9e1f-a468eb5983b7', metadata={'id': 23}, page_content='Navigates to the issue screen to view a list of reported tasks or issues.')]